## ⚠️ Per chi viene da Pandas/ML

Se conosci **Pandas**, Spark è simile ma con differenze chiave:
- **Distribuito**: i dati sono su più macchine, non su una sola
- **Lazy**: Spark non esegue subito, pianifica ed esegue quando serve
- **Immutabile**: ogni trasformazione crea un nuovo DataFrame

### 🔑 Equivalenze rapide:
```python
# Pandas → Spark
df.head()               → df.show()
df.info()               → df.printSchema()
len(df)                 → df.count()
df[df['age'] > 18]      → df.filter(col('age') > 18)
df[['col1', 'col2']]    → df.select('col1', 'col2')
df.groupby('x').sum()   → df.groupBy('x').sum()
```

### ⛔ MAI fare:
```python
# ❌ NON convertire a Pandas per operazioni su grandi dataset
df.toPandas().groupby('col').sum()  # SBAGLIATO: porta tutto in memoria!

# ✅ USA Spark distribuito
df.groupBy('col').sum()  # CORRETTO: resta distribuito
```

📘 **Guida completa**: Vedi [PANDAS_TO_SPARK.ipynb](PANDAS_TO_SPARK.ipynb) per capire le differenze in dettaglio.

---

# 🔧 Databricks Cheatsheet - In ordine di complessità

## 📊 1. Lettura Dati Base

In [ ]:
# Lettura file CSV
df = spark.read.csv("path/to/file.csv", header=True, inferSchema=True)

# Lettura JSON
df = spark.read.json("path/to/file.json")

# Lettura Parquet
df = spark.read.parquet("path/to/file.parquet")

# Da tabella
df = spark.read.table("database.table_name")

## 🔍 2. Esplorazione Base

In [ ]:
# Info struttura
df.printSchema()    # Schema colonne
df.show(5)          # Prime 5 righe
df.count()          # Totale righe
df.columns          # Lista colonne

# Statistiche base
df.describe().show()
df.select("column_name").distinct().show()

## 🎯 3. Selezione e Filtri

In [ ]:
# Selezione colonne
df.select("col1", "col2").show()

# Filtri base
df.filter(df.age > 18).show()
df.filter((df.age > 18) & (df.city == "Rome")).show()
df.where(df.status == "active").show()

# Limit
df.limit(100).show()

## 🔄 4. Trasformazioni Base

In [ ]:
from pyspark.sql.functions import *

# Nuove colonne
df.withColumn("age_plus_10", col("age") + 10).show()
df.withColumn("constant", lit("value")).show()

# Rinominare
df.withColumnRenamed("old_name", "new_name").show()

# Ordinamento
df.orderBy("age").show()
df.orderBy(desc("age")).show()

# Drop colonne
df.drop("unwanted_col").show()

## 📊 5. Aggregazioni

In [ ]:
# Group By base
df.groupBy("category").count().show()
df.groupBy("category").sum("amount").show()
df.groupBy("category").avg("price").show()

# Aggregazioni multiple
df.groupBy("category").agg(
    count("*").alias("total_records"),
    sum("amount").alias("total_amount"),
    avg("price").alias("avg_price")
).show()

## 🔗 6. Join Base

In [ ]:
# Join tipi
df1.join(df2, "common_column").show()           # Inner
df1.join(df2, "common_column", "left").show()   # Left
df1.join(df2, "common_column", "right").show()  # Right
df1.join(df2, "common_column", "outer").show()  # Full outer

# Join con colonne diverse
df1.join(df2, df1.id == df2.user_id).show()

## 💾 7. Salvataggio Base

In [ ]:
# Salva tabella
df.write.saveAsTable("my_database.my_table")

# Salva file
df.write.parquet("path/to/output.parquet")
df.write.csv("path/to/output.csv", header=True)

# Modalità scrittura
df.write.mode("overwrite").saveAsTable("table")
df.write.mode("append").saveAsTable("table")

## 🗂️ 8. SQL Magic

In [ ]:
# Registra come vista
df.createOrReplaceTempView("my_data")

# Usa SQL
result = spark.sql("SELECT * FROM my_data WHERE age > 25")

# Query complesse
spark.sql("""
    SELECT category, COUNT(*) as count, AVG(price) as avg_price
    FROM my_data 
    GROUP BY category
    ORDER BY avg_price DESC
""").show()

## 🗄️ 9. Unity Catalog - Database/Tabelle

In [ ]:
# Creare database
spark.sql("CREATE DATABASE IF NOT EXISTS my_database")
spark.sql("USE my_database")

# Creare tabella da DataFrame
df.write.saveAsTable("my_database.customers")

# Creare tabella vuota
spark.sql("""
    CREATE TABLE IF NOT EXISTS my_database.orders (
        order_id INT,
        customer_id INT,
        order_date DATE,
        amount DECIMAL(10,2)
    )
""")

# Info tabelle
spark.sql("SHOW TABLES IN my_database").show()
spark.sql("DESCRIBE my_database.customers").show()

## 🔺 10. Delta Lake Operations

In [ ]:
from delta.tables import DeltaTable

# Salva come Delta
df.write.format("delta").saveAsTable("my_database.delta_table")

# Merge (UPSERT)
delta_table = DeltaTable.forName(spark, "my_database.customers")
delta_table.alias("target").merge(
    new_data.alias("source"),
    "target.customer_id = source.customer_id"
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

# Time Travel
spark.sql("SELECT * FROM my_database.customers VERSION AS OF 1").show()

# Ottimizzazioni
spark.sql("OPTIMIZE my_database.customers")
spark.sql("OPTIMIZE my_database.customers ZORDER BY (customer_id)")
spark.sql("VACUUM my_database.customers")

## 🔗 11. Storage Mounting

In [ ]:
# Mount Azure Data Lake (ADLS)
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "your_client_id",
    "fs.azure.account.oauth2.client.secret": "your_client_secret",
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/your_tenant_id/oauth2/token"
}

dbutils.fs.mount(
    source="abfss://container@storageaccount.dfs.core.windows.net/",
    mount_point="/mnt/datalake",
    extra_configs=configs
)

# Mount S3
dbutils.fs.mount(
    source="s3a://your-bucket-name",
    mount_point="/mnt/s3bucket",
    extra_configs={
        "fs.s3a.access.key": "your_access_key",
        "fs.s3a.secret.key": "your_secret_key"
    }
)

# Gestione mount
dbutils.fs.mounts()                    # Lista mount
dbutils.fs.unmount("/mnt/datalake")   # Unmount

## 🚀 12. Performance Tips (SOLO SPARK!)

In [ ]:
# ❌ MAI usare pandas!
# ❌ SBAGLIATO: df.toPandas().groupby('col').sum()
# ✅ CORRETTO: df.groupBy('col').sum()

# Cache per performance
df.cache()    # Mantiene in memoria
df.persist()  # Più controllo
df.unpersist() # Rimuove cache

# Broadcast join (tabelle < 200MB)
from pyspark.sql.functions import broadcast
large_df.join(broadcast(small_df), "key").show()

# Partizionamento
df.repartition(200)  # Ridistribuisce
df.coalesce(50)      # Riduce partizioni

# Configurazioni
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.coalescePartitions.enabled", "true")

## 🔄 13. ETL Patterns Avanzati

In [ ]:
# Window Functions
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank

window_spec = Window.partitionBy("customer_id").orderBy(desc("order_date"))
df.withColumn("row_num", row_number().over(window_spec)).show()

# Deduplicazione
df.dropDuplicates(["customer_id", "email"]).show()

# Slowly Changing Dimension (SCD Type 2)
# Pattern complesso per tracking cambiamenti storici
def scd_type2_update(target_table, source_df, key_cols):
    # Chiudi record vecchi, inserisci nuovi
    # Implementazione complessa per produzione
    pass

## 📝 14. Template Esercizio Standard

In [ ]:
# TEMPLATE STANDARD PER OGNI ESERCIZIO

# 1. Leggi i dati
df = spark.read.csv("path/to/data.csv", header=True, inferSchema=True)

# 2. Esplora sempre prima!
df.printSchema()
df.show(5)
print(f"Righe totali: {df.count()}")

# 3. Pulisci se necessario
df_clean = df.filter(df.column.isNotNull())

# 4. Trasforma
df_transformed = df_clean.withColumn("new_col", col("old_col") * 2)

# 5. Analizza
result = df_transformed.groupBy("category").agg(
    count("*").alias("count"),
    avg("value").alias("avg_value")
)

# 6. Mostra risultato
result.show()

# 7. Salva se richiesto
# result.write.saveAsTable("my_database.results")